In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import shutil
import assimilation
import os

rng = np.random.default_rng()

In [ ]:
# space
x = np.linspace(0, 10, 100)
# total time to integrate over
ts = np.linspace(0, 10, 1000)
# observation timesteps
obs_steps1 = np.arange(2, 1000, 50)
obs_steps2 = np.arange(250, 750, 25)
obs_steps3 = np.arange(375, 625, 12)
# Derivative order
alpha = 1.1

# observation mode
mode = 'boolean'
# observation matrix, a truth table if boolean
H  = np.ones(100).astype(int) == 1
#H[49:]=0
#H = H==1

# ensemble members
n_members = 40

# Operator in Fourier space
def L_k (u, ks):
  k_v = ks[np.newaxis, :, np.newaxis]
  return (-k_v*k_v)*u
def L_k2 (u, ks):
  k_v = ks[np.newaxis, :, np.newaxis]
  return (-k_v*k_v - np.power(1j*k_v, alpha))*u

# initial truth
z = np.exp(-(x-5)*(x-5)/0.3)

# background covariance matrix, 1d array if boolean
B = np.ones(100) * 1

# observation covariance matrix, 1d array if boolean
P = np.ones(np.sum(H)) * 10

In [ ]:
(z_true, z_mean1, z_cov1) = assimilation.assimilation(z, x, ts, obs_steps1, alpha, L_k, n_members, H, P, B)
(z_true, z_mean2, z_cov2) = assimilation.assimilation(z, x, ts, obs_steps2, alpha, L_k, n_members, H, P, B)
(z_true, z_mean3, z_cov3) = assimilation.assimilation(z, x, ts, obs_steps3, alpha, L_k, n_members, H, P, B)
(z_true, z_mean_none, z_cov_none) = assimilation.assimilation(z, x, ts, [], alpha, L_k, n_members, H, P, B)

In [ ]:
z_cov_total1 = np.sum(np.diagonal(z_cov1), axis=1)
z_cov_total2 = np.sum(np.diagonal(z_cov2), axis=1)
z_cov_total3 = np.sum(np.diagonal(z_cov3), axis=1)
z_cov_none_total = np.sum(np.diagonal(z_cov_none), axis=1)
fig, ax = plt.subplots()
ax.plot(ts, z_cov_total1)
ax.plot(ts, z_cov_total2)
ax.plot(ts, z_cov_total3)
ax.set_xlabel('Time')
ax.set_ylabel('Total variance')
#ax.scatter(ts[obs_steps], np.zeros(obs_steps.shape), marker='|', c='r')
plt.savefig('var-2nd-1_1', dpi=300)
fig, ax = plt.subplots()
ax.plot(ts, z_cov_total1/z_cov_none_total)
ax.plot(ts, z_cov_total2/z_cov_none_total)
ax.plot(ts, z_cov_total3/z_cov_none_total)
ax.set_xlabel('Time')
ax.set_ylabel('Relative variance')
plt.savefig('rel-var-2nd-1_1', dpi=300)

In [ ]:
assimilation.animation('anim', x, z_mean[:, ::5], z_cov=z_cov[:, :, ::5])